In [161]:
import pandas as pd
import matplotlib.pyplot as plt
from konlpy.tag import Mecab
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import numpy as np
from pykrx import stock
import datetime
import numpy as np
import matplotlib
import seaborn as sns
import time

In [162]:
def tokenizer(text):
    text = Mecab().nouns(text)
    return [word for word in text if len(word)>1]

class topic_stock():
    def __init__(self,start,end,etf_code,n_top_words=5):
        news = pd.read_excel(f'.\\파이썬+텍스트+분석+입문\\파이썬 텍스트 분석 입문\\data\\NewsResult_{str(start)}-{str(end)}.xlsx')
        news = news.dropna(axis=1)
        self.news = news
        self.start = start
        self.end = end
        # 1년단위로 한다면 뉴스의 양이 많아 부정확할수 있으므로 3개월 정도의 뉴스를 추천한다
        data = []
        for date in news['일자'].unique():
            d_news = news.loc[self.news['일자'] == date,'본문'] # 불리언 인덱싱 방식으로 두개의 칼럼을 인덱싱가능하다.
            '''날짜당 기사가 10개 미만이라면 dateNews에 추가하지 않는다
            그러면 ohlcv와 길이가 맞지 않으므로 ohlcv와 
            dateNews['date']를 merge시켜준다.'''
            if len(d_news)>10:
                # 본문 내용의 끝에 '기사 끝'이란 단어를 추가해 붙여줌
                text = '기사 끝'.join(d_news.to_list())
                # 같은 날짜의 기사본문 추가
                data.append({'date':date,'text':text})

        self.dateNews = pd.DataFrame(data)
        self.dateNews['date'] = pd.to_datetime(self.dateNews['date'].map(str))

        # vectorizer의 기본 설정
        self.vectorizer = TfidfVectorizer(tokenizer=tokenizer,
                            max_df=0.90,
                            min_df=2,
                            max_features=5000)
        dateW_matrix = self.vectorizer.fit_transform(self.dateNews['text'])

        nmf = NMF(n_components=20, alpha=0.2, random_state=3)
        self.dateT = nmf.fit_transform(dateW_matrix)
        self.TW = nmf.components_

        # # 옵션 1 : 주말장의 값은 미래장의 값으로 채운다.
        # self.stock_ohlcv = stock.get_etf_ohlcv_by_date(self.start,self.end,etf_code)
        # self.stock_ohlcv = pd.merge(self.stock_ohlcv,pd.Series(index=self.dateNews['date'],name='dummy')
        #                             ,how='outer',left_index=True,right_index=True)
        # self.stock_ohlcv = self.stock_ohlcv.fillna(method='bfill')
        # self.stock_ohlcv.dropna(how='all',axis='columns',inplace=True) # col내 모든 값이 nan이면 없애기

        # topic마다 상위 빈도 5개 단어를 self.topic에 저장
        self.topic = []
        for topic_idx, word_vec in enumerate(self.TW):
            message = "Topic %d: " % topic_idx

            message += " ".join(["#"+self.vectorizer.get_feature_names()[i]
                         for i in word_vec.argsort()[:-n_top_words-1:-1]])
            self.topic.append(message)

    def stock_ohlcv(self,etf_code):
        # 옵션 1 : 주말장의 값은 미래장의 값으로 채운다.    
        stock_ohlcv = stock.get_etf_ohlcv_by_date(self.start,self.end,etf_code)
        stock_ohlcv = pd.merge(stock_ohlcv,pd.Series(index=self.dateNews['date'],name='dummy')
                            ,how='outer',left_index=True,right_index=True)
        stock_ohlcv = stock_ohlcv.fillna(method='bfill')
        stock_ohlcv.dropna(how='all',axis='columns',inplace=True) # col내 모든 값이 nan이면 없애기
        return stock_ohlcv

    def print_top_words(self):
        '''토픽과 그단어를 출력'''
        print('\n'.join(self.topic))

    def topicid_to_topic(self, topic_id):
        label = " ".join([self.vectorizer.get_feature_names()[i]
                        for i in self.TW[topic_id].argsort()[:-6:-1]])
        return label

    def get_etf_ticker_list(self):
        '''기간 내에 ohlcv가 존재하는 etf의 ticker 반환'''
        tickers_a = stock.get_etf_ticker_list(str(self.end))
        tickers_b = stock.get_etf_ticker_list(str(self.start))
        etf_code_list = list(set(tickers_a) & set(tickers_b)) # 분석 시작과 끝의 날짜에 둘다 존재하는 ticker 찾기
        return etf_code_list

    def print_load_top_doc(self, topic_id, n_top_words=5, n_top_titles=10):
        '''토픽과 가장 관련된 기사의 본문과 제목 출력
        dateT에서 topic과 가장 연관된 date에 본문,제목 5개 출력
        너무 기니까 각기사 안의 토픽의 빈도수가 가장 높은 문서 5개를 출력해보자'''

        # topic_id에 가장관련된 한 날짜의 본문들을 기사 끝이라는 것으로 분할해 리스트로 만든다. '기사 끝'으로 구분된 기사 본문을 리스트로 만들기
        text_list = self.dateNews.iloc[self.dateT.T[topic_id].argsort()[-1],1].split('기사 끝')
        text_list_df = pd.DataFrame(text_list,columns=['본문'])

        vectorizer = TfidfVectorizer(tokenizer=tokenizer,
                            max_df=0.90,
                            min_df=2,
                            max_features=1000,
                            )
        # 한 날짜의 본문들을 벡터화한다.
        DW_matrix = vectorizer.fit_transform(text_list_df['본문'])
        
        nmf = NMF(n_components=3)
        DT = nmf.fit_transform(DW_matrix)
        TW = nmf.components_

        # text_list_df로 분석했으므로 인덱스 유지를 위해 오른쪽대상의 키('본문'칼럼)만을 기준으로(how='right',on='본문') merge
        topic_news = pd.merge(self.news, text_list_df, how='right', on='본문')

        data = ['{0}{1}와 가장 밀접한 날짜의 뉴스분석{0}'.format('-'*10,self.topic[topic_id])]
        
        # 토픽 단어 5개 + 기사 제목 + 기사 url
        for topic_idx, (doc_vec, word_vec) in enumerate(zip(DT.T, TW)):
            message = "\nTopic %d: " % topic_idx
            # 토픽 단어    
            message += "#"+" #".join([vectorizer.get_feature_names()[i]
                        for i in word_vec.argsort()[:-n_top_words - 1:-1]])
            
            message += '\n\n'
            # 기사 제목(index = 3), 기사 url(index = 8)
            message += "\n".join([topic_news.iloc[i,3]+'\n'+topic_news.iloc[i,8]
                        for i in doc_vec.argsort()[:-n_top_titles - 1:-1]])

            message += "\n\n"
            data.append(message)
            print(message)

        f = open("C:\\Users\\JAEHO\\Desktop\\textanalysis.txt", 'w')
        f.write('\n'.join(data))
        f.close()


    def graph1(self, n_topics, etf_code,비교항목):
        '''x축을 공유하는 subplot 2개를 그린다.'''
        sns.set_style("darkgrid")
        matplotlib.rcParams['font.family'] = 'NanumGothic'

        fig, (ax1,ax2) = plt.subplots(2,1,sharex=True) #squeeze = True라 일차원이 된다.
        for topic_id in range(n_topics):
            label = "Topic #%d:\n" % topic_id
            label += " ".join([self.vectorizer.get_feature_names()[i]
                             for i in self.TW[topic_id].argsort()[:-6:-1]])
    
            h = self.dateT / self.dateT.sum(1, keepdims=True)
            h = h[:, topic_id].ravel()

            ax1.plot(self.dateNews['date'], h)
            ax1.fill_between(self.dateNews['date'], h, alpha=0.5, label=label)
            ax1.legend(fontsize=10, bbox_to_anchor=(0.1, 0.2))

        stock_ohlcv = self.stock_ohlcv(etf_code)    
        ax2.plot(stock_ohlcv.index,stock_ohlcv[비교항목],label='주가')
        ax2.set_ylabel('원')
        ax2.legend(fontsize=10)

        plt.show()

    def graph2(self, start_topic, end_topic, 비교항목):
        fig, ax1 = plt.subplots() #squeeze = True라 일차원이 된다.

        count = range(20)[start_topic:end_topic]
        for topic_id in count:
            label = "Topic #%d:\n" % topic_id
            label += " ".join([self.vectorizer.get_feature_names()[i]
                             for i in self.TW[topic_id].argsort()[:-6:-1]])
    
            h = self.dateT / self.dateT.sum(1, keepdims=True)
            h = h[:, topic_id].ravel()

            ax1.plot(self.dateNews['date'], h)
            ax1.fill_between(self.dateNews['date'], h, alpha=0.5, label=label)
            ax1.legend(fontsize=10, bbox_to_anchor=(0.1, 0.2))


        ax2 = ax1.twinx()
        ax2.plot(self.stock_ohlcv.index,self.stock_ohlcv[비교항목],'r--',label=비교항목)
        ax2.set_ylabel('원')
        ax2.legend(fontsize=10)

        plt.show()

    def topic_stock_corr(self):
        # dateT와 주식 df를 합친다.
        etf_code_list = self.get_etf_ticker_list()
        corr_df = pd.DataFrame(columns=['종목명','관련 토픽 번호','토픽단어','연관수치'])
        for etf_code in etf_code_list[:100]:
            stock_ohlcv = self.stock_ohlcv(etf_code)
            dateT = pd.DataFrame(self.dateT,index=stock_ohlcv.index)
            df = pd.merge(dateT,stock_ohlcv,how='outer',left_index=True,right_index=True)
            corr = df.corr(method='pearson')
            corr = corr.iloc[:20,20:]
            series = corr.loc[corr['거래량'].argsort()[::-1],'거래량']
            df = pd.DataFrame({'종목명':stock.get_etf_ticker_name(etf_code),'관련 토픽 번호':series.index,
                            '토픽단어':series.index.map(self.topicid_to_topic),'연관수치':series.to_list()})
            corr_df = pd.concat([corr_df,df])
            time.sleep(0.3)
        corr_df = corr_df[(corr_df['연관수치']>0.3)|(corr_df['연관수치']<-0.3)]
        corr_df = corr_df.sort_values(by='관련 토픽 번호')
        return corr_df
    

In [163]:
a=topic_stock('20201128','20211128','305720')

C:\Users\JAEHO\anaconda3\envs\py37-64\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\JAEHO\anaconda3\envs\py37-64\lib\site-packages\sklearn\decomposition\_nmf.py:315: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  "'nndsvda' in 1.1 (renaming of 0.26)."), FutureWarning)
C:\Users\JAEHO\anaconda3\envs\py37-64\lib\site-packages\sklearn\decomposition\_nmf.py:1091: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)


In [164]:
b=a.topic_stock_corr()

C:\Users\JAEHO\anaconda3\envs\py37-64\lib\site-packages\ipykernel_launcher.py:58: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [185]:
b.sort_values(by='관련 토픽 번호')

,종목명,관련 토픽 번호,토픽단어,연관수치
0,TIGER 2차전지테마,0,아스트라 영국 러시아 허가 변이,0.318627
0,KODEX 미국달러선물,0,아스트라 영국 러시아 허가 변이,0.325057
0,KBSTAR 헬스케어채권혼합,1,위드 일상 회복 청원 단계,0.650738
0,KODEX 미국달러선물인버스,1,위드 일상 회복 청원 단계,0.388734
0,KBSTAR 200선물인버스2X,1,위드 일상 회복 청원 단계,0.405017
0,KODEX 혁신기술테마액티브,1,위드 일상 회복 청원 단계,0.505928
0,KODEX 코스닥150롱코스피200숏선물,1,위드 일상 회복 청원 단계,0.362670
0,KOSEF 200선물인버스,1,위드 일상 회복 청원 단계,0.386161
0,KODEX 코스피100,1,위드 일상 회복 청원 단계,0.480174
0,TIGER 헬스케어,1,위드 일상 회복 청원 단계,0.594554
